<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
Cross FAQ Conversations
</div>

## Initial

In [37]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
OPENROUTER_API_KEY = os.getenv("SHISHA_API_KEY")
GROQCLOUD_API_KEY = os.getenv("GROQCLOUD_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [38]:
import requests
from openai import OpenAI 

OPENROUTER_CLIENT = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

OPENAI_CLIENT = OpenAI(api_key=OPENAI_API_KEY)

def call_openai_api(prompt: str) -> str:
    try:
        response = OPENAI_CLIENT.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
        )

        reply = response.choices[0].message.content

        return reply 
    
    except Exception as e:
        print(f"API called failed: {e}")
        return None

def call_openrouter_api(prompt: str) -> str:
    try:
        completion = OPENROUTER_CLIENT.chat.completions.create(
            model="moonshotai/kimi-dev-72b:free",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )

        return completion.choices[0].message.content
    
    except Exception as e:
        print(f"API called failed: {e}")
        return None
    
def call_groq_api(prompt: str) -> str:
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {GROQCLOUD_API_KEY}"
    }
    payload = {
        "model": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        
        return result["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"GROQ API call failed: {e}")
        return None

In [39]:
CONVERSATION_PROMPT = """You are a helpful hotel assistant. Based on the 3 Q&A pairs below, generate a natural multi-turn conversation. 
Make sure that: 
- The first mention must includes the name of hotel.
- Use **short, simple, concise questions** (like a user might type quickly), and keep the flow natural. 
- Do not expand questions into long sentences. The assistant answers should remain informative but brief.

Return the output as a list of alternating messages between human and gpt in JSON format like this:
[
    {{ "from": "human", "value": "..." }},
    {{ "from": "gpt", "value": "..." }},
    ...
]

Here are the 3 Q&A:
{merged}
"""

## Top-3 question-answer

In [40]:
import json

file_path = "./0-preprocessing.json"
list_questions = []
list_answers = []
list_hotels_name = []

with open(file_path, 'r', encoding='utf-8') as file:
    raw_data = json.load(file)

for data in raw_data:
    list_questions.append(data['conversations'][0]['value'])
    list_answers.append(data['conversations'][1]['value'])
    list_hotels_name.append(data['hotel_name'])

for question, answer, hotel_name in zip(list_questions[:3], list_answers[:3], list_hotels_name[:3]):
    print(hotel_name)
    print(question)
    print(answer)
    print("\n===\n")

Renaissance Riverside Saigon Hotel
Bạn có thể cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel được không?
Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham quan nổi tiếng như Nhà hát Lớn, Chợ Bến Thành.

===

Renaissance Riverside Saigon Hotel
Khách sạn Renaissance Riverside Saigon nằm ở địa chỉ nào vậy?
Renaissance Riverside Saigon Hotel tọa lạc tại địa chỉ 8 - 15, Ton Duc Thang, Quận 1, Thành phố Hồ Chí Minh, Việt Nam.

===

Renaissance Riverside Saigon Hotel
Bạn có thể cho tôi biết các hạng phòng hiện có tại Renaissance Riverside Saigon Hotel không?
Khách sạn Renaissance Riverside Saigon có nhiều hạng phòng đa dạng như: Standard Room, Suite, Guest Room, Club Guest Room, Executive Studio, Executive Suite, R Club Room, D

In [46]:
from collections import defaultdict
import random
from typing import Dict, List, Tuple

def group_by_hotel(data):
    grouped = defaultdict(list)

    for object in data:
        hotel_name = object['hotel_name']
        question = object['conversations'][0]['value']
        answer = object['conversations'][1]['value']
        grouped[hotel_name].append((question, answer))

    return grouped

def merge_faq_pairs(conversations: List[Tuple]) -> str:
    merged = []
    for question, answer in conversations:
        merged.append(f"Q: {question}\nA: {answer}")
    return "\n\n".join(merged)

def run_augmentation_pipeline(data: List[Dict], repeat=50):
    grouped_hotels = group_by_hotel(data)
    all_augmented = []

    for hotel_name, conversations in grouped_hotels.items():
        print(conversations)
        if len(conversations) < 3:
            print(f"Hotel {hotel_name} has less than 3 conversations, skipping.")
            continue

        for _ in range(repeat):
            selected_conversations = random.sample(conversations, 3)
            merged_conversations = merge_faq_pairs(selected_conversations)
            prompt = CONVERSATION_PROMPT.format(merged=merged_conversations)

            response = call_openai_api(prompt)
            print(response)

            try:
                cross_faq_conversation = json.loads(response)
                all_augmented.append({
                    "hotel_name": hotel_name,
                    "conversations": cross_faq_conversation
                })
                print(f"🦉 Current length of all_augmented: {len(all_augmented)}")

            except json.JSONDecodeError as e:
                print(f"Failed to parse JSON response: {e}")
                continue

    return all_augmented

In [47]:
list_conversations = run_augmentation_pipeline(raw_data, repeat=75)


[('Bạn có thể cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel được không?', 'Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham quan nổi tiếng như Nhà hát Lớn, Chợ Bến Thành.'), ('Khách sạn Renaissance Riverside Saigon nằm ở địa chỉ nào vậy?', 'Renaissance Riverside Saigon Hotel tọa lạc tại địa chỉ 8 - 15, Ton Duc Thang, Quận 1, Thành phố Hồ Chí Minh, Việt Nam.'), ('Bạn có thể cho tôi biết các hạng phòng hiện có tại Renaissance Riverside Saigon Hotel không?', 'Khách sạn Renaissance Riverside Saigon có nhiều hạng phòng đa dạng như: Standard Room, Suite, Guest Room, Club Guest Room, Executive Studio, Executive Suite, R Club Room, Deluxe Room, Deluxe Suite, Superior Room, Executive Room, và Presidential Suite.'), ('Bạn hãy li

In [48]:
list_conversations[:2]

[{'hotel_name': 'Renaissance Riverside Saigon Hotel',
  'conversations': [{'from': 'human',
    'value': 'Chào Renaissance Riverside Saigon, có phòng Presidential Suite không?'},
   {'from': 'gpt',
    'value': 'Có, khách sạn có phòng Presidential Suite – hạng phòng cao cấp nhất với tiện nghi sang trọng.'},
   {'from': 'human', 'value': 'Có phòng Executive Studio không?'},
   {'from': 'gpt',
    'value': 'Có, nằm tại Club level, phòng Studio 1 King Bed, rộng rãi và cao cấp.'},
   {'from': 'human', 'value': 'Có những hạng phòng nào khác?'},
   {'from': 'gpt',
    'value': 'Chúng tôi có Standard Room, Suite, Guest Room, Club Guest Room, Executive Suite, R Club Room, Deluxe Room, Superior Room, Executive Room, và Presidential Suite.'}]},
 {'hotel_name': 'Renaissance Riverside Saigon Hotel',
  'conversations': [{'from': 'human',
    'value': 'Cần đặt phòng ở khách sạn Renaissance Riverside Saigon.'},
   {'from': 'gpt',
    'value': 'Chào bạn! Bạn muốn biết về các loại phòng tại Renaissance

## Save to file

In [49]:
def save_to_file(data: List[Dict], filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("[")

        for index, entry in enumerate(data):
            char = ','
            if index == len(data) - 1:
                char = ''

            file.write(json.dumps(entry, ensure_ascii=False) + f"{char}\n")

        file.write("]")
        
    print(f'Saved {len(data)} entries to {filename}')

In [50]:
save_to_file(list_conversations, '3-cross_faq_conversations.json')

Saved 720 entries to 3-cross_faq_conversations.json
